# <center>Household Power Consumption Prediction<center/>

### Description 
This notebook implements a an sequential model from Household Power Consumption Prediction. Model input are measurements of electric power consumption. Data was gathered from one household with a one-minute sampling rate over a period of almost 4 years.

#### Data Set Information:

The dataset contains 2075259 samples/measurements gathered between December 2006 and November 2010 (47 months).<br>
**NOTE**: 
> (global_active_power*1000/60 - sub_metering_1 - sub_metering_2 - sub_metering_3) represents the active energy consumed every minute (in watt hour) in the household by electrical equipment not measured in sub-meterings 1, 2 and 3.

> The dataset contains some missing values in the measurements (nearly 1,25% of the rows). All calendar timestamps are present in the dataset but for some timestamps, the measurement values are missing: a missing value is represented by the absence of value between two consecutive semi-colon attribute separators. For instance, the dataset shows missing values on April 28, 2007.

**Data Attributes Information:** 

1. date: Date in format dd/mm/yyyy

2. time: time in format hh:mm:ss

3. global_active_power: household global minute-averaged active power (in kilowatt)

4. global_reactive_power: household global minute-averaged reactive power (in kilowatt)

5. voltage: minute-averaged voltage (in volt)

6. global_intensity: household global minute-averaged current intensity (in ampere)

7. sub_metering_1: energy sub-metering No. 1 (in watt-hour of active energy). It corresponds to the kitchen, containing mainly a dishwasher, an oven and a microwave (hot plates are not electric but gas powered).

8. sub_metering_2: energy sub-metering No. 2 (in watt-hour of active energy). It corresponds to the laundry room, containing a washing-machine, a tumble-drier, a refrigerator and a light.

9. sub_metering_3: energy sub-metering No. 3 (in watt-hour of active energy). It corresponds to an electric water-heater and an air-conditioner. <br>

#### Task : 
> Predict the power consumption together with upper and lower interval

### Import necessary libraries 

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm

import lightgbm as lgb
from catboost import CatBoostRegressor

### Read data

In [ ]:
data_path = "../Dataset/household_power_consumption_data.zip"
df = pd.read_csv(data_path, sep=';',parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True,
                 low_memory=False, na_values=['nan','?'],index_col='dt')
df.drop(["Global_active_power","Global_reactive_power","Voltage","Global_intensity"],axis=1,inplace=True)

### Preprocess Data
Fill NaN values by average, extract features from timestamps & group the data

In [ ]:
#fill nan values with column average
for j in range(0,3):
    df.iloc[:,j]=df.iloc[:,j].fillna(df.iloc[:,j].mean())

df["consumption"] = df.iloc[:,:].sum(axis=1)
df.head()

In [ ]:
#Extract features from timestamp 
def features_from_timestamp(t):
    h = t.hour
    year = t.year
    idx = np.searchsorted(list(range(3,25,3)),h,side='right')
    interval = np.arange(3,25,3)[idx]
    if  interval == 24 : interval = 0
    month = t.month
    season = (month in [11,12,1,2,3]) * 1 # 0: summer, 1:  winter
    return [t.day, t.dayofweek,h,interval,month,season]


col = ['day', 'dayofweek', 'hour', 'interval', 'month', 'season']

In [ ]:
grouped = df["consumption"].groupby(pd.Grouper(freq='1h', base=0, label='right')).sum()
data = pd.DataFrame(grouped/60)

#Get intervals

# Generate aditional features from timestamp then append to exixting data
additional_featues = pd.DataFrame(data = [features_from_timestamp(i) for i in tqdm(data.index) ],columns=col).set_index(data.index)
data = data.merge(additional_featues,on="dt")
data.sort_index(inplace=True) #make sure data is sorted by date

for col in list(data.columns)[1:]:
    data[col] = data[col].astype("category")
cate_features_name = ['day', 'dayofweek', 'hour', 'interval', 'month', 'season']

data.head()

### Create dataset with upper bound and lower bound
For each record look back one month and calculate the standard deviation, quantile (5% and 95%) and the mean energy consumption. Check if each record is between the upper bound and lower bound (5 and 95 percent quantile in this case)

In [ ]:
data2 = pd.DataFrame(grouped/60)
def get_intervals(t):
    global data2
    lookback = t - pd.Timedelta('730hr')
    day_name = t.day_name()
    mask = ((data2.index >= lookback) & (data2.index <= t))

    if np.sum(mask) < 144:
        return [np.nan, np.nan,np.nan,np.nan]

    d = data[mask].groupby(data[mask].index.weekday_name)
#     temp = d.get_group(day_name).values
    temp = d.get_group(day_name)
    temp = temp.groupby(temp.index.hour)
    k = t.hour if len(temp.groups.keys()) == 24 else t.hour - 1
    if k == -1 :
        k = 23
    
    temp = temp.get_group(k).values

    p1 = np.quantile(temp,0.05)
    p2 = np.quantile(temp,0.95)
    
    return [temp.std() ,temp.mean(), p1, p2]
    

In [ ]:
add_col = ["std","mean","p1","p2"]
addi = pd.DataFrame(data = [get_intervals(i) for i in tqdm(data2.index)],columns=add_col).set_index(data2.index)
data2 = data2.merge(addi,on="dt")
data2.dropna(inplace=True)
data2['in_bounds'] = data2["consumption"].between(left=data2['p1'], right=data2['p2'])

In [ ]:
data2.head()

In [ ]:
data2.in_bounds.value_counts(normalize=True)

### Split data to train and test
Take one year as test sample (2010) and 3 years as trainning set (2006 - 2009)

In [ ]:
#Split data to train and test
xtrain = data.loc["2006":"2010"]
ytrain = xtrain.pop("consumption")

xtest = data.loc["2010":]
ytest = xtest.pop("consumption")

### Define models & Train models
Define and set modes training paramaters

#### Use sklearn Gradient Boosting Regressor

In [ ]:
LOWER_ALPHA = 0.10
UPPER_ALPHA = 0.90

N_ESTIMATORS = 700
MAX_DEPTH = 5

# Each model has to be separate
lower_model = GradientBoostingRegressor(loss="quantile", alpha=LOWER_ALPHA, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)
# The mid model will use the default loss
mid_model = GradientBoostingRegressor(loss="ls", n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)
upper_model = GradientBoostingRegressor(loss="quantile", alpha=UPPER_ALPHA, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)


#Train Models
_ = lower_model.fit(xtrain, ytrain)
_ = mid_model.fit(xtrain, ytrain)
_ = upper_model.fit(xtrain, ytrain)

In [ ]:
mse = mean_squared_error(mid_model.predict(xtest),ytest)
print("Gradient Boosting Regressor MSE : {}".format(mse))

#### Use LightGBM

In [ ]:
# params = {"task":"train","max_depth": -1, "objective": "quantile","learning_rate" : 0.1,
#           "alpha":0.5 ,"num_leaves": 900,'metric': {'l2', 'auc'},  "n_estimators": 700,
#          "n_jobs" : -1}

# d_train = lgb.Dataset(xtrain, label=ytrain, categorical_feature = cate_features_name)
# model2 = lgb.train(params, d_train)

lower_model_lgbm = lgb.LGBMRegressor(objective="quantile",alpha=0.05,n_estimators=700,max_depth=15,n_jobs=-1, learning_rate= 0.1,
                                   num_leaves= 900)
mid_model_lgbm = lgb.LGBMRegressor(objective="quantile",alpha=0.5,n_estimators=700,max_depth=15,n_jobs=-1, learning_rate= 0.1,
                                   num_leaves= 900)
upper_model_lgbm = lgb.LGBMRegressor(objective="quantile",alpha=0.95,n_estimators=700,max_depth=15,n_jobs=-1, learning_rate= 0.1,
                                     num_leaves= 900)
#Train Models
lower_model_lgbm.fit(xtrain, ytrain,verbose = 0,categorical_feature=cate_features_name,eval_metric = ['l2', 'auc'])
mid_model_lgbm.fit(xtrain, ytrain,verbose = 0,categorical_feature=cate_features_name,eval_metric = ['l2', 'auc'])
upper_model_lgbm.fit(xtrain, ytrain,verbose = 0,categorical_feature=cate_features_name,eval_metric = ['l2', 'auc'])


In [ ]:
mse = mean_squared_error(mid_model_lgbm.predict(xtest),ytest)
print("LightGBM MSE : {}".format(mse))

### Predictions

In [ ]:
predictions = pd.DataFrame(ytest)
predictions['lower'] = lower_model_lgbm.predict(xtest)
predictions['mid'] = mid_model_lgbm.predict(xtest)
predictions['upper'] = upper_model_lgbm.predict(xtest)

assert (predictions['upper'] > predictions['lower']).all()

predictions.tail()

In [ ]:
#Clip negative values to zero
predictions.loc[predictions['lower'] < 0.0, "lower"] = 0.0
predictions.loc[predictions['mid'] < 0.0, "mid"] = 0.0

### Calculate model Error

In [ ]:
def calculate_error(predictions):
    predictions['absolute_error_lower'] = (predictions['lower'] - predictions["consumption"]).abs()
    predictions['absolute_error_upper'] = (predictions['upper'] - predictions["consumption"]).abs()
    
    predictions['absolute_error_interval'] = (predictions['absolute_error_lower'] + predictions['absolute_error_upper']) / 2
    predictions['absolute_error_mid'] = (predictions['mid'] - predictions["consumption"]).abs()
    
    predictions['in_bounds'] = predictions["consumption"].between(left=predictions['lower'], right=predictions['upper'])
    

In [ ]:
calculate_error(predictions)
metrics = predictions[['absolute_error_lower', 'absolute_error_upper', 'absolute_error_interval', 'absolute_error_mid', 'in_bounds']].copy()
metrics.describe()

In [ ]:
predictions.in_bounds.value_counts(normalize=True)

### Plot model predictions

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
n = 100
fig = plt.figure(figsize=(19.20,10.80))
# plt.plot(predictions.index[100:150], predictions.lower[100:150],label="Lower",linewidth=3)
plt.plot(predictions.index[:n], predictions.mid[:n],label="Prediction",linewidth=3)
# plt.plot(predictions.index[100:150], predictions.upper[100:150], label="Upper",linewidth=3)
plt.fill_between(predictions.index[:n], predictions.lower[:n], predictions.upper[:n],
                 alpha=0.3, facecolor=colors[8],label='interval')
plt.plot(predictions.index[:n], predictions.consumption[:n],label="actual",linewidth=3)
plt.legend(loc=0,prop={'size': 20})
plt.ylabel("Energy Consumption (Wh)",fontsize=20.0,fontweight="bold")
plt.xlabel("Time",fontsize=20.0,fontweight="bold")
plt.tick_params(labelsize=15)
plt.show()

### Exploratory Data Analysis (optional)

In [ ]:
# import pandas_profiling
# profile = pandas_profiling.ProfileReport(df)
# profile.to_file("report.html")
# profile

### <center> Predictions of intervals using Deep Quantile Regression (Deep Neural network) <center/>

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
import keras.backend as K

In [ ]:
#scale data
scaler = StandardScaler()
std_xtrain = scaler.fit_transform(xtrain.values)
std_xtest = scaler.transform(xtest.values)

In [ ]:
def quantile_loss(q,y,f):
    # q: Quantile to be evaluated, e.g., 0.5 for median.
    # y: True value.
    # f: Fitted (predicted) value.
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)


In [ ]:
def BaseModel():
    model = Sequential()
    model.add(Dense(units=10, input_dim=6,activation='relu'))
    model.add(Dense(units=10, input_dim=1,activation='relu'))
    model.add(Dense(1))
    
    return model

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

model_lower = BaseModel()
model_lower.compile(loss=lambda y,f: quantile_loss(0.1,y,f), optimizer='adadelta')
model_lower.fit(std_xtrain, ytrain, epochs=200, batch_size=32, verbose=0,validation_split=0.2, callbacks=[early_stop])

model_middle = BaseModel()
model_middle.compile(loss=lambda y,f: quantile_loss(0.5,y,f), optimizer='adadelta')
model_middle.fit(std_xtrain, ytrain, epochs=200, batch_size=32, verbose=0,validation_split=0.2, callbacks=[early_stop])

model_upper = BaseModel()
model_upper.compile(loss=lambda y,f: quantile_loss(0.9,y,f), optimizer='adadelta')
model_upper.fit(std_xtrain, ytrain, epochs=200, batch_size=32, verbose=0,validation_split=0.2, callbacks=[early_stop])
